In [455]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import os, sys
sys.path.append('../../src/')

import toolbox

In [456]:
# List 
# individual coins: btc, ada, eth, ltc, xrp
# indexes: BITX, BITW20, BITW100, CRIX

index = ['BITX', 'BITW20', 'BITW70', 'BITW100', 'CRIX']
coin_arr = ['ada', 'eth', 'ltc', 'xrp']

In [457]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as ss
import seaborn as sns
import sys, os
sys.path.append('../../src/')
from toolbox import *
import datetime

import string

result_names = ['BBT_Tiingo',                #0
                'BBT_future_Tiingo_ada', 
                'BBT_future_Tiingo_eth',     #2
                'BBT_future_Tiingo_ltc',
                'BBT_future_Tiingo_xrp',     #4
                'BBT_future_BITX',
                'BBT_future_BITW20',         #6
                'BBT_future_BITW70', 
                'BBT_future_BITW100',        #8
                'BBT_future_CRIX']

risk_measures = ['Variance',
                 'ES q=0.05', 'ES q=0.01',
                 'VaR q=0.05', 'VaR q=0.01',
                 'ERM k=10', 'CH']

order = ['BBT_Tiingo',    
         'BBT_future_CRIX', 'BBT_future_BITX', 'BBT_future_BITW100', 
         'BBT_future_BITW20',  'BBT_future_BITW70',
         'BBT_future_Tiingo_ada', 'BBT_future_Tiingo_eth',  'BBT_future_Tiingo_ltc','BBT_future_Tiingo_xrp']

coin_names = ['btc', 'CRIX', 'BITX', 'BITW100', 'BITW20', 'BITW70', 'ada', 'eth', 'ltc', 'xrp']

new_coin_names = ['BTC', 'CRIX', 'BITX', 'BITW100', 'BITW20', 'BITW70', 'ADA', 'ETH', 'LTC', 'XRP']

start = []
for n in range(len(order)):
    rn = order[n]
    rm = 'ES q=0.05'
    path = '../../results/%s/MM/rh_%s_copula_selection.csv'\
                    %(rn, rm)
    
    df = pd.read_csv(path)
    df.Date = pd.to_datetime(df.Date)
    
    start.append(np.min(df.Date))
    
start = np.max(start)

start = '2019-10-21'

## Table1

In [532]:
# Table 1a: Moments, dependency with future, start and end
def summary_statistics(df, coin_name, start='2000-01-01'):
    
    df = df.loc[df.Date >= start,:]
    
    rs = df.loc[:,'log return %s'%coin_name]
    rf = df.loc[:, 'log return future']
    
    m1 = np.mean(rs)
    m2 = np.std(rs)
    m3 = ss.skew(rs) # Pearson coefficient of skewness
    m4 = ss.kurtosis(rs) # Pearson coefficient of kurtosis

    _max = np.max(rs)
    _min = np.min(rs)
    
    _min_day = df.iloc[np.argmin(rs), :].Date[:10]

    tau = ss.kendalltau(rs,rf)[0]
    rho = np.corrcoef(rs, rf)[0,1]

    end = df.Date.iloc[0]
    start = df.Date.iloc[-1]
    
    _dict = {'m1':m1, 'm2':m2, 'm3':m3, 'm4':m4, 
#             'max':_max, 
             'min':_min, 'min_day':_min_day, 
            'tau':tau, 'rho':rho}#, 
#             'start':start[:10], 'end':end[:10]}
    
    return pd.DataFrame([_dict], [coin_name])

In [533]:
# Table 1a: Moments, dependency with future, start and end
def summary_statistics_rh(df, coin_name, start='2000-01-01'):
    
    df = df.loc[df.Date >= start,:]
    
    rh = df.rh
    
    m1 = np.mean(rh)
    m2 = np.std(rh)
    m3 = ss.skew(rh) # Pearson coefficient of skewness
    m4 = ss.kurtosis(rh) # Pearson coefficient of kurtosis

    _max = np.max(rh)
    _min = np.min(rh)
    
    _min_day = df.iloc[np.argmin(rh), :].Date[:10]

    end = df.Date.iloc[0]
    start = df.Date.iloc[-1]
    
    _dict = {'m1':m1, 'm2':m2, 'm3':m3, 'm4':m4, 
#             'max':_max, 
             'min':_min, 'min_day':_min_day}#, 
#             'start':start[:10], 'end':end[:10]}
    
    return pd.DataFrame([_dict], [coin_name])

In [534]:
coin_arr = ['ada', 'eth', 'ltc', 'xrp']
table1 = pd.DataFrame()
for c in coin_arr:
    coin_name = c
    file_name = 'BBT_future_Tiingo_'+coin_name+'.csv'
    path = '../../data/cleaned_data/'+file_name
    df = pd.read_csv(path)
    df = df.sort_values('Date', ascending=False)
    table1 = table1.append(summary_statistics(df, c, start))
    

In [535]:
table1

,m1,m2,m3,m4,min,min_day,rho,tau
ada,0.009467,0.066990,0.166123,2.308589,-0.268528,2020-03-12,0.629635,0.482461
eth,0.006819,0.060103,-0.255677,5.264618,-0.320144,2020-03-12,0.771182,0.598765
ltc,0.003227,0.064781,-0.993548,5.301061,-0.375913,2021-05-19,0.808022,0.611283
xrp,0.002987,0.079843,0.554200,12.488167,-0.527652,2020-12-23,0.451039,0.493882


In [536]:
# btc
path = '../../data/cleaned_data/BBT_Tiingo.csv'
df = pd.read_csv(path)

table1 = table1.append(summary_statistics(df, 'bitcoin', start))
table1 = table1.append(summary_statistics(df, 'future', start))

In [537]:
for index_name in index:
    file_name = 'BBT_future_'+index_name+'.csv'
    path = '../../data/cleaned_data/'+file_name
    df = pd.read_csv(path)
    df = df.sort_values('Date', ascending=False)
    table1 = table1.append(summary_statistics(df, index_name, start))

In [538]:
table1

,m1,m2,m3,m4,min,min_day,rho,tau
ada,0.009467,0.066990,0.166123,2.308589,-0.268528,2020-03-12,0.629635,0.482461
eth,0.006819,0.060103,-0.255677,5.264618,-0.320144,2020-03-12,0.771182,0.598765
ltc,0.003227,0.064781,-0.993548,5.301061,-0.375913,2021-05-19,0.808022,0.611283
xrp,0.002987,0.079843,0.554200,12.488167,-0.527652,2020-12-23,0.451039,0.493882
bitcoin,0.003915,0.044023,-0.585724,4.656525,-0.259965,2020-03-12,0.997513,0.950675
future,0.003906,0.046312,-0.505986,4.420370,-0.269920,2020-03-12,1.000000,1.000000
BITX,0.004308,0.045676,-0.884202,4.722154,-0.270220,2020-03-12,0.976873,0.873785
BITW20,0.006249,0.055021,-1.151786,5.220325,-0.310092,2020-03-12,0.767431,0.588253
BITW70,0.006353,0.058155,-1.117070,5.192645,-0.323453,2021-05-19,0.752537,0.545908
BITW100,0.004683,0.046174,-0.986368,4.938073,-0.272694,2020-03-12,0.967404,0.853713


In [539]:
_index = ['btc','eth','ada','ltc','xrp',
          'BITX','CRIX','BITW100','BITW20','BITW70']

table1.iloc[:,[0,1,4]]*=100

In [540]:
table1 = np.round(table1,4)

In [541]:
table1.columns = ['Mean %', 'Std %', 'Skew', 'Kurt', 'MD %',
                  'MD date', '\\rho', '\\tau']

In [543]:
table1

,Mean %,Std %,Skew,Kurt,MD %,MD date,\rho,\tau
ada,0.9467,6.6990,0.1661,2.3086,-26.8528,2020-03-12,0.6296,0.4825
eth,0.6819,6.0103,-0.2557,5.2646,-32.0144,2020-03-12,0.7712,0.5988
ltc,0.3227,6.4781,-0.9935,5.3011,-37.5913,2021-05-19,0.8080,0.6113
xrp,0.2987,7.9843,0.5542,12.4882,-52.7652,2020-12-23,0.4510,0.4939
bitcoin,0.3915,4.4023,-0.5857,4.6565,-25.9965,2020-03-12,0.9975,0.9507
future,0.3906,4.6312,-0.5060,4.4204,-26.9920,2020-03-12,1.0000,1.0000
BITX,0.4308,4.5676,-0.8842,4.7222,-27.0220,2020-03-12,0.9769,0.8738
BITW20,0.6249,5.5021,-1.1518,5.2203,-31.0092,2020-03-12,0.7674,0.5883
BITW70,0.6353,5.8155,-1.1171,5.1926,-32.3453,2021-05-19,0.7525,0.5459
BITW100,0.4683,4.6174,-0.9864,4.9381,-27.2694,2020-03-12,0.9674,0.8537


In [544]:
_index = ['future','bitcoin',
          'eth','ada','ltc','xrp',
          'BITX','CRIX','BITW100',
          'BITW20','BITW70']


_index_name = ['BTCF', 'BTC', 'ETH', 'ADA', 'LTC', 'XRP', 
               'BITX', 'CRIX', 'BITW100', 'BITW20', 'BITW70']


In [545]:
table1 = table1.reindex(_index)
# table2 = table2.reindex(_index)

table1

In [547]:
table1.index= _index_name
table1

,Mean %,Std %,Skew,Kurt,MD %,MD date,\rho,\tau
BTCF,0.3906,4.6312,-0.5060,4.4204,-26.9920,2020-03-12,1.0000,1.0000
BTC,0.3915,4.4023,-0.5857,4.6565,-25.9965,2020-03-12,0.9975,0.9507
ETH,0.6819,6.0103,-0.2557,5.2646,-32.0144,2020-03-12,0.7712,0.5988
ADA,0.9467,6.6990,0.1661,2.3086,-26.8528,2020-03-12,0.6296,0.4825
LTC,0.3227,6.4781,-0.9935,5.3011,-37.5913,2021-05-19,0.8080,0.6113
XRP,0.2987,7.9843,0.5542,12.4882,-52.7652,2020-12-23,0.4510,0.4939
BITX,0.4308,4.5676,-0.8842,4.7222,-27.0220,2020-03-12,0.9769,0.8738
CRIX,0.4602,4.5420,-0.7952,4.7549,-27.1385,2020-03-12,0.9799,0.8769
BITW100,0.4683,4.6174,-0.9864,4.9381,-27.2694,2020-03-12,0.9674,0.8537
BITW20,0.6249,5.5021,-1.1518,5.2203,-31.0092,2020-03-12,0.7674,0.5883


In [570]:
table1_latex = table1.iloc[:,:].to_latex()
# print(table1_latex)
table1_latex_arr = table1_latex.split('\n')

In [571]:
table1_latex_arr[4]

'BTCF    &  0.3906 &  4.6312 & -0.5060 &   4.4204 & -26.9920 &  2020-03-12 &  1.0000 &  1.0000 \\\\'

In [572]:
table1_latex_arr[4]

'BTCF    &  0.3906 &  4.6312 & -0.5060 &   4.4204 & -26.9920 &  2020-03-12 &  1.0000 &  1.0000 \\\\'

In [578]:
file = open("../../latex/_tables/summaryStatistics.tex", "r")

lineNumberToReplace = np.array([6,8,9,10,11,12,14,15,16,18,19])-1
infoLineNumber = [4,5,6,7,8,9,10,11,12,13,14]

newContent = ""

filelines = [line for line in file]
file.close()

In [579]:
for i in range(len(lineNumberToReplace)):
    filelines[lineNumberToReplace[i]] = '\\ \\ \\ '+ table1_latex_arr[infoLineNumber[i]]+'\n'

In [580]:
toWrite = ''.join(filelines)

fout = open("../../latex/_tables/summaryStatistics.tex", "w")
fout.write(toWrite)
fout.close()

## Table 2
### ERM

In [478]:
# BTC
n = 0
rn = order[n]
rm = 'ES q=0.05'
path = '../../results/%s/MM/rh_%s_copula_selection.csv'\
                %(rn, rm)

df = pd.read_csv(path)
df.Date = pd.to_datetime(df.Date)

In [479]:
ERM_table = pd.DataFrame()

rm = 'ERM k=10'


for i, rn in enumerate(order):
    path = '../../results/%s/MM/rh_%s_copula_selection.csv'\
                    %(rn, rm)

    df = pd.read_csv(path)
#     df.Date = pd.to_datetime(df.Date)
    df = df.sort_values('Date', ascending=False)
    
    row = summary_statistics_rh(df, new_coin_names[i], start)
    
    row.loc[:,rm]=toolbox.ERM_estimate_trapezoidal(10, df.rh)
    
    ERM_table = ERM_table.append(row)

ERM_table.columns = ['Mean %', 'Std %', 'Skew', 'Kurt', 'MD %',
                  'MD date', rm]

ERM_table.iloc[:,[0,1,4]] *= 100

ERM_table = np.round(ERM_table, 4)

ERM_table

,Mean %,Std %,Skew,Kurt,MD %,MD date,ERM k=10
BTC,0.0223,0.3221,-1.0008,3.4153,-1.5242,2020-11-30,0.0057
CRIX,0.0840,0.9087,0.0488,14.5501,-7.0530,2021-05-19,0.0100
BITX,0.0590,1.0078,-0.4427,13.0839,-7.8581,2021-05-19,0.0127
BITW100,0.0853,1.2032,-1.6522,20.5562,-11.1846,2021-05-19,0.0153
BITW20,0.2564,3.6009,-0.3446,4.2152,-21.5920,2021-05-19,0.0503
BITW70,0.2818,3.9074,-0.6952,4.8745,-24.5250,2021-05-19,0.0557
ADA,0.5722,5.3590,1.4203,4.6970,-14.3885,2021-01-08,0.0700
ETH,0.3117,3.8679,1.0345,7.5751,-18.8729,2021-05-19,0.0491
LTC,-0.0512,3.8812,-0.2929,7.7022,-28.0879,2021-05-19,0.0616
XRP,0.0155,7.1579,1.1244,19.8583,-52.5689,2020-12-23,0.0787


In [443]:
_index_name = ['BTC', 'ETH', 'ADA', 'LTC', 'XRP', 
               'BITX', 'CRIX', 'BITW100', 'BITW20', 'BITW70']
ERM_table = ERM_table.reindex(_index_name)

ERM_table_latex = ERM_table.iloc[:,:].to_latex()
# print(table1_latex)
ERM_table_latex_arr = ERM_table_latex.split('\n')

file = open("../../latex/_tables/ERM_rh.tex", "r")

lineNumberToReplace = np.array([6,7,8,9,10,12,13,14,16,17])-1
infoLineNumber = [4,5,6,7,8,9,10,11,12,13]

newContent = ""

filelines = [line for line in file]
file.close()

for i in range(len(lineNumberToReplace)):
    filelines[lineNumberToReplace[i]] = '\\ \\ \\ '+ ERM_table_latex_arr[infoLineNumber[i]]+'\n'
    
toWrite = ''.join(filelines)

fout = open("../../latex/_tables/ERM_rh.tex", "w")
fout.write(toWrite)
fout.close()

In [377]:
VaR5_table = pd.DataFrame()

rm = 'VaR q=0.05'

for i, rn in enumerate(order):
    path = '../../results/%s/MM/rh_%s_copula_selection.csv'\
                    %(rn, rm)

    df = pd.read_csv(path)
#     df.Date = pd.to_datetime(df.Date)
    df = df.sort_values('Date', ascending=False)
    
    row = summary_statistics_rh(df, new_coin_names[i], start)
    
    row.loc[:,rm]=toolbox.VaR(0.05, df.rh)
    
    VaR5_table = VaR5_table.append(row)

VaR5_table.columns = ['Mean %',  'Std %', 'Skew', 'Kurt', 'MD %',
                      'MD date', 'VaR 5%']

VaR5_table.iloc[:,[0,1,4]] *= 100

VaR5_table = np.round(VaR5_table, 4)

VaR5_table

,Mean %,Std %,Skew,Kurt,MD %,MD date,VaR 5%
BTC,0.0253,0.3294,-0.9725,3.4373,-1.5347,2020-11-30,0.0063
CRIX,0.0863,0.9151,0.0718,13.7915,-6.9744,2021-05-19,0.0092
BITX,0.0562,0.9930,-0.3117,12.4780,-7.5639,2021-05-19,0.0128
BITW100,0.0846,1.1980,-1.6592,21.3725,-11.2582,2021-05-19,0.0164
BITW20,0.2728,3.5940,-0.3721,4.4896,-22.0733,2021-05-19,0.0546
BITW70,0.2847,3.9133,-0.6580,4.7874,-24.6513,2021-05-19,0.0626
ADA,0.5726,5.2204,1.2981,4.2544,-14.6974,2021-05-19,0.0769
ETH,0.3084,3.8944,1.0243,7.4297,-19.1750,2021-05-19,0.0514
LTC,-0.0742,3.9145,-0.3836,7.5384,-28.3672,2021-05-19,0.0622
XRP,0.0208,7.1520,1.1269,19.8930,-52.5667,2020-12-23,0.0683


In [444]:
_index_name = ['BTC', 'ETH', 'ADA', 'LTC', 'XRP', 
               'BITX', 'CRIX', 'BITW100', 'BITW20', 'BITW70']
VaR5_table = VaR5_table.reindex(_index_name)

VaR5_table_latex = VaR5_table.iloc[:,:].to_latex()
# print(table1_latex)
VaR5_table_latex_arr = VaR5_table_latex.split('\n')

file = open("../../latex/_tables/VaR5_rh.tex", "r")

lineNumberToReplace = np.array([6,7,8,9,10,12,13,14,16,17])-1
infoLineNumber = [4,5,6,7,8,9,10,11,12,13]

newContent = ""

filelines = [line for line in file]
file.close()

for i in range(len(lineNumberToReplace)):
    filelines[lineNumberToReplace[i]] = '\\ \\ \\ '+ VaR5_table_latex_arr[infoLineNumber[i]]+'\n'
    
toWrite = ''.join(filelines)

fout = open("../../latex/_tables/VaR5_rh.tex", "w")
fout.write(toWrite)
fout.close()

In [381]:
VaR1_table = pd.DataFrame()

rm = 'VaR q=0.01'

for i, rn in enumerate(order):
    path = '../../results/%s/MM/rh_%s_copula_selection.csv'\
                    %(rn, rm)

    df = pd.read_csv(path)
    df = df.sort_values('Date', ascending=False)
    
    row = summary_statistics_rh(df, new_coin_names[i], start)
    
    row.loc[:,rm]=toolbox.VaR(0.01, df.rh)
    
    VaR1_table = VaR1_table.append(row)

VaR1_table.columns = ['Mean %', 'Std %', 'Skew', 'Kurt', 'MD %',
                  'MD date', 'VaR 1%']

VaR1_table.iloc[:,[0,1,4]] *= 100

VaR1_table = np.round(VaR1_table, 4)

VaR1_table

,Mean %,Std %,Skew,Kurt,MD %,MD date,VaR 1%
BTC,0.0176,0.3270,-1.0405,3.3742,-1.5689,2020-11-30,0.0134
CRIX,0.0738,0.9695,-0.4729,13.6500,-7.0185,2021-05-19,0.0245
BITX,0.0593,1.0178,-0.5331,13.3100,-8.0299,2021-05-19,0.0247
BITW100,0.0823,1.2338,-1.9365,23.1938,-11.8752,2021-05-19,0.0347
BITW20,0.2499,3.6210,-0.3866,4.3396,-21.6634,2021-05-19,0.0988
BITW70,0.2788,3.9257,-0.7635,5.1288,-24.5294,2021-05-19,0.1147
ADA,0.5562,5.3466,1.1362,3.9334,-15.4795,2021-05-19,0.1106
ETH,0.2977,3.9132,0.9547,7.2414,-18.6061,2021-05-19,0.1026
LTC,-0.0852,4.1503,-0.7234,7.3208,-29.0915,2021-05-19,0.1030
XRP,0.0352,7.1658,1.1582,19.8506,-52.5727,2020-12-23,0.1387


In [445]:
_index_name = ['BTC', 'ETH', 'ADA', 'LTC', 'XRP', 
               'BITX', 'CRIX', 'BITW100', 'BITW20', 'BITW70']
VaR1_table = VaR1_table.reindex(_index_name)

VaR1_table_latex = VaR1_table.iloc[:,:].to_latex()
# print(table1_latex)
VaR1_table_latex_arr = VaR1_table_latex.split('\n')

file = open("../../latex/_tables/VaR1_rh.tex", "r")

lineNumberToReplace = np.array([6,7,8,9,10,12,13,14,16,17])-1
infoLineNumber = [4,5,6,7,8,9,10,11,12,13]

filelines = [line for line in file]
file.close()

for i in range(len(lineNumberToReplace)):
    filelines[lineNumberToReplace[i]] = '\\ \\ \\ '+ VaR1_table_latex_arr[infoLineNumber[i]]+'\n'
    
toWrite = ''.join(filelines)

fout = open("../../latex/_tables/VaR1_rh.tex", "w")
fout.write(toWrite)
fout.close()

In [382]:
ES5_table = pd.DataFrame()

rm = 'ES q=0.05'

for i, rn in enumerate(order):
    path = '../../results/%s/MM/rh_%s_copula_selection.csv'\
                    %(rn, rm)

    df = pd.read_csv(path)
    df = df.sort_values('Date', ascending=False)
    
    row = summary_statistics_rh(df, new_coin_names[i], start)
    
    row.loc[:,rm]=toolbox.ES(0.05, df.rh)
    
    ES5_table = ES5_table.append(row)

ES5_table.columns = ['Mean %', 'Std %', 'Skew', 'Kurt', 'MD %',
                  'MD date', 'ES 5%']

ES5_table.iloc[:,[0,1,4]] *= 100

ES5_table = np.round(ES5_table, 4)

ES5_table

,Mean %,Std %,Skew,Kurt,MD %,MD date,ES 5%
BTC,0.0204,0.3234,-1.0150,3.4423,-1.5629,2020-11-30,0.0101
CRIX,0.0777,0.9207,0.0164,13.5608,-6.9894,2021-05-19,0.0173
BITX,0.0591,1.0065,-0.3453,12.1335,-7.6211,2021-05-19,0.0215
BITW100,0.0848,1.2125,-1.6397,19.7472,-11.1357,2021-05-19,0.0274
BITW20,0.2608,3.6115,-0.3555,4.2016,-21.5430,2021-05-19,0.0804
BITW70,0.2785,3.9157,-0.6949,4.8047,-24.3474,2021-05-19,0.0908
ADA,0.5525,5.2673,1.2557,4.2423,-14.9647,2021-05-19,0.0984
ETH,0.3082,3.8890,1.0119,7.4077,-18.7819,2021-05-19,0.0782
LTC,-0.0808,3.9829,-0.4957,7.2302,-28.4608,2021-05-19,0.0962
XRP,0.0176,7.1533,1.1411,19.9176,-52.5698,2020-12-23,0.1354


In [447]:
_index_name = ['BTC', 'ETH', 'ADA', 'LTC', 'XRP', 
               'BITX', 'CRIX', 'BITW100', 'BITW20', 'BITW70']
ES5_table = ES5_table.reindex(_index_name)

ES5_table_latex = ES5_table.iloc[:,:].to_latex()
# print(table1_latex)
ES5_table_latex_arr = ES5_table_latex.split('\n')

file = open("../../latex/_tables/ES5_rh.tex", "r")

lineNumberToReplace = np.array([6,7,8,9,10,12,13,14,16,17])-1
infoLineNumber = [4,5,6,7,8,9,10,11,12,13]

filelines = [line for line in file]
file.close()

for i in range(len(lineNumberToReplace)):
    filelines[lineNumberToReplace[i]] = '\\ \\ \\ '+ ES5_table_latex_arr[infoLineNumber[i]]+'\n'
    
toWrite = ''.join(filelines)

fout = open("../../latex/_tables/ES5_rh.tex", "w")
fout.write(toWrite)
fout.close()

In [395]:
ES1_table = pd.DataFrame()

rm = 'ES q=0.01'

for i, rn in enumerate(order):
    path = '../../results/%s/MM/rh_%s_copula_selection.csv'\
                    %(rn, rm)

    df = pd.read_csv(path)
    df = df.sort_values('Date', ascending=False)
    
    row = summary_statistics_rh(df, new_coin_names[i], start)
    
    row.loc[:,rm]=toolbox.ES(0.01, df.rh)
    
    ES1_table = ES1_table.append(row)

ES1_table.columns = ['Mean %', 'Std %', 'Skew', 'Kurt', 'MD %',
                  'MD date', 'ES 1%']

ES1_table.iloc[:,[0,1,4]] *= 100

# ES1_table = np.round(ES1_table, 4)

ES1_table

,Mean %,Std %,Skew,Kurt,MD %,MD date,ES 1%
BTC,0.014808,0.347559,-0.835432,3.305439,-1.622527,2020-11-30,0.023384
CRIX,0.083524,0.946138,-0.036149,12.404693,-7.020300,2021-05-19,0.034979
BITX,0.059801,1.031183,-0.440995,11.586333,-7.742446,2021-05-19,0.041121
BITW100,0.078145,1.263996,-1.964529,21.883557,-11.926340,2021-05-19,0.059302
BITW20,0.253793,3.632266,-0.408622,4.446241,-21.986552,2021-05-19,0.128208
BITW70,0.266023,3.932037,-0.759783,5.004985,-24.476390,2021-05-19,0.153457
ADA,0.501591,5.404018,1.100767,3.960739,-15.448083,2021-05-19,0.146283
ETH,0.307977,3.895367,0.984009,7.494711,-18.762453,2021-05-19,0.129950
LTC,-0.102863,4.158086,-0.775700,7.437497,-29.172721,2021-05-19,0.164734
XRP,-0.019965,7.288724,1.112146,18.873229,-52.569956,2020-12-23,0.251630


In [449]:
_index_name = ['BTC', 'ETH', 'ADA', 'LTC', 'XRP', 
               'BITX', 'CRIX', 'BITW100', 'BITW20', 'BITW70']
ES1_table = ES1_table.reindex(_index_name)

ES1_table_latex = ES1_table.iloc[:,:].to_latex()
# print(table1_latex)
ES1_table_latex_arr = ES1_table_latex.split('\n')

file = open("../../latex/_tables/ES1_rh.tex", "r")

lineNumberToReplace = np.array([6,7,8,9,10,12,13,14,16,17])-1
infoLineNumber = [4,5,6,7,8,9,10,11,12,13]

filelines = [line for line in file]
file.close()

for i in range(len(lineNumberToReplace)):
    filelines[lineNumberToReplace[i]] = '\\ \\ \\ '+ ES1_table_latex_arr[infoLineNumber[i]]+'\n'
    
toWrite = ''.join(filelines)

fout = open("../../latex/_tables/ES1_rh.tex", "w")
fout.write(toWrite)
fout.close()

In [386]:
var_table = pd.DataFrame()

rm = 'Variance'

for i, rn in enumerate(order):
    path = '../../results/%s/MM/rh_%s_copula_selection.csv'\
                    %(rn, rm)

    df = pd.read_csv(path)
    df = df.sort_values('Date', ascending=False)
    
    row = summary_statistics_rh(df, new_coin_names[i], start)
    
    row.loc[:,rm]=toolbox.Variance(df.rh)
    
    var_table = var_table.append(row)

var_table.columns = ['Mean %', 'Std %', 'Skew', 'Kurt', 'MD %',
                  'MD date', 'Variance']

var_table.iloc[:,[0,1,4]] *= 100

var_table = np.round(var_table, 4)

var_table

,Mean %,Std %,Skew,Kurt,MD %,MD date,Variance
BTC,0.0215,0.3221,-1.0119,3.1929,-1.4393,2020-11-30,0.0000
CRIX,0.0812,0.9183,-0.0027,14.3136,-7.1025,2021-05-19,0.0001
BITX,0.0561,0.9954,-0.4204,13.2487,-7.7567,2021-05-19,0.0001
BITW100,0.0855,1.1986,-1.7440,22.2644,-11.3866,2021-05-19,0.0001
BITW20,0.2429,3.5846,-0.3063,4.1622,-21.4680,2021-05-19,0.0013
BITW70,0.2706,3.8838,-0.6490,4.6312,-23.9984,2021-05-19,0.0015
ADA,0.5617,5.2722,1.3634,4.4818,-13.8687,2021-01-08,0.0028
ETH,0.2823,3.8741,0.9469,7.1064,-17.7421,2021-05-19,0.0015
LTC,-0.0871,3.9052,-0.3617,7.6239,-28.3029,2021-05-19,0.0018
XRP,-0.0123,7.1537,1.1451,20.0236,-52.5236,2020-12-23,0.0043


In [452]:
_index_name = ['BTC', 'ETH', 'ADA', 'LTC', 'XRP', 
               'BITX', 'CRIX', 'BITW100', 'BITW20', 'BITW70']
var_table = var_table.reindex(_index_name)

var_table_latex = var_table.iloc[:,:].to_latex()
# print(table1_latex)
var_table_latex_arr = var_table_latex.split('\n')

file = open("../../latex/_tables/var_rh.tex", "r")

lineNumberToReplace = np.array([6,7,8,9,10,12,13,14,16,17])-1
infoLineNumber = [4,5,6,7,8,9,10,11,12,13]

filelines = [line for line in file]
file.close()

for i in range(len(lineNumberToReplace)):
    filelines[lineNumberToReplace[i]] = '\\ \\ \\ '+ var_table_latex_arr[infoLineNumber[i]]+'\n'
    
toWrite = ''.join(filelines)

fout = open("../../latex/_tables/var_rh.tex", "w")
fout.write(toWrite)
fout.close()

## Combine


In [588]:
summaryTables = ['var_rh.tex', 'VaR5_rh.tex', 'VaR1_rh.tex', 'ES5_rh.tex', 'ES1_rh.tex', 'ERM_rh.tex']

filelines = ['\section{Summary Statistics of Hedged Portfolios}\label{sec:SSHP}', '\n']

path = '../../latex/_tables/'
for sT in summaryTables:
    file = open(path+sT, "r")
    filelines+=[line for line in file]+['']
    file.close()

toWrite = ''.join(filelines)

fout = open("../../latex/_tables/combinedSummaryStatistics.tex", "w")
fout.write(toWrite)
fout.close()


In [589]:
filelines

['\\section{Summary Statistics of Hedged Portfolios}\\label{sec:SSHP}',
 '\n',
 '\\begin{table}[!] \\centering \\resizebox{\\textwidth}{!}{%\n',
 '\\begin{tabular}{lrrrrrrr} \\toprule\n',
 '         {} &    Mean \\% &     Std \\% &      Skew &       Kurt &         MD \\% &     MD date & Variance \\\\\n',
 '\\midrule\n',
 '     \\multicolumn{7}{l}{Individual Cryptos}                                                                                 \\\\\n',
 '\\ \\ \\ BTC     &  0.0215 &  0.3221 & -1.0119 &   3.1929 &  -1.4393 &  2020-11-30 &    0.0000 \\\\\n',
 '\\ \\ \\ ETH     &  0.2823 &  3.8741 &  0.9469 &   7.1064 & -17.7421 &  2021-05-19 &    0.0015 \\\\\n',
 '\\ \\ \\ ADA     &  0.5617 &  5.2722 &  1.3634 &   4.4818 & -13.8687 &  2021-01-08 &    0.0028 \\\\\n',
 '\\ \\ \\ LTC     & -0.0871 &  3.9052 & -0.3617 &   7.6239 & -28.3029 &  2021-05-19 &    0.0018 \\\\\n',
 '\\ \\ \\ XRP     & -0.0123 &  7.1537 &  1.1451 &  20.0236 & -52.5236 &  2020-12-23 &    0.0043 \\\\\n',
 '   \\multi

## BTC

In [582]:
order

['BBT_Tiingo',
 'BBT_future_CRIX',
 'BBT_future_BITX',
 'BBT_future_BITW100',
 'BBT_future_BITW20',
 'BBT_future_BITW70',
 'BBT_future_Tiingo_ada',
 'BBT_future_Tiingo_eth',
 'BBT_future_Tiingo_ltc',
 'BBT_future_Tiingo_xrp']

In [608]:
BTC_table = pd.DataFrame()

risk_measure_names = ['Variance',
                      'ES 95%', 'ES 99%',
                      'VaR 95%', 'VaR 99%',
                      'ERM k=10', 'CH']

rn = 'BBT_Tiingo'

for i, rm in enumerate(risk_measures[:-1]):
    path = '../../results/%s/MM/rh_%s_copula_selection.csv'\
                    %(rn, rm)

    df = pd.read_csv(path)
#     df.Date = pd.to_datetime(df.Date)
    df = df.sort_values('Date', ascending=False)
    
    row = summary_statistics_rh(df, risk_measure_names[i], start)
    
#     row.loc[:,rm]=toolbox.VaR(0.05, df.rh)
    
    BTC_table = BTC_table.append(row)

risk_measures_order =  ['Variance',
                  'VaR 95%', 'VaR 99%',
                  'ES 95%', 'ES 99%',
                  'ERM k=10']

BTC_table = BTC_table.reindex(risk_measures_order)
BTC_table.iloc[:,:2] *= 100

BTC_table = np.round(BTC_table, 4)
BTC_table.columns = ['Mean %', 'Std %', 'Skew', 'Kurt', 'MD %', 'MD date']


In [617]:
BTC_table

,Mean %,Std %,Skew,Kurt,MD %,MD date
Variance,0.0215,0.3221,-1.0119,3.1929,-0.0144,2020-11-30
VaR 95%,0.0253,0.3294,-0.9725,3.4373,-0.0153,2020-11-30
VaR 99%,0.0176,0.3270,-1.0405,3.3742,-0.0157,2020-11-30
ES 95%,0.0204,0.3234,-1.0150,3.4423,-0.0156,2020-11-30
ES 99%,0.0148,0.3476,-0.8354,3.3054,-0.0162,2020-11-30
ERM k=10,0.0223,0.3221,-1.0008,3.4153,-0.0152,2020-11-30


In [616]:
BTC_table_latex = BTC_table.iloc[:,:].to_latex()
# print(table1_latex)
BTC_table_latex_arr = BTC_table_latex.split('\n')

file = open("../../latex/_tables/BTC_rh.tex", "r")

lineNumberToReplace = np.array([7,8,9,10,11,12])-1
infoLineNumber = [4,5,6,7,8,9]

filelines = [line for line in file]
file.close()

for i in range(len(lineNumberToReplace)):
    filelines[lineNumberToReplace[i]] = BTC_table_latex_arr[infoLineNumber[i]]+'\n'
    
toWrite = ''.join(filelines)

fout = open("../../latex/_tables/BTC_rh.tex", "w")
fout.write(toWrite)
fout.close()

## Other Stuffs

In [451]:
def risk_statistics(df, coin_name, start='2000-01-01'):
    
    df = df.loc[df.Date >= start,:]

    rs = df.loc[:,'log return %s'%coin_name]
    ERM = toolbox.ERM_estimate_trapezoidal(10, rs)
    ES1 = toolbox.ES(0.01, rs)
    ES5 = toolbox.ES(0.05, rs)
    VaR1 = toolbox.VaR(0.01, rs)
    VaR5 = toolbox.VaR(0.05, rs)
    Var = toolbox.Variance(rs)
    
    _dict = {'ERM':ERM, 'ES1':ES1, 'ES5':ES5, 'VaR1':VaR1, 
             'VaR5':VaR5, 'Var':Var}
    
    return pd.DataFrame([_dict], [coin_name])

In [123]:
table2 = pd.DataFrame()

for c in coin_arr:
    coin_name = c
    file_name = 'BBT_future_Tiingo_'+coin_name+'.csv'
    path = '../../data/cleaned_data/'+file_name
    df = pd.read_csv(path)
    df = df.sort_values('Date', ascending=False)
    table2 = table2.append(risk_statistics(df, c, '2018-08-01'))
    
path = '../../data/cleaned_data/BBT_Tiingo.csv'
df = pd.read_csv(path)

table2 = table2.append(risk_statistics(df, 'bitcoin'))
table2 = table2.append(risk_statistics(df, 'future'))

for index_name in index:
    file_name = 'BBT_future_'+index_name+'.csv'
    path = '../../data/cleaned_data/'+file_name
    df = pd.read_csv(path)
    df = df.sort_values('Date', ascending=False)
    table2 = table2.append(risk_statistics(df, index_name, '2018-08-01'))

In [151]:
# table2 = table2.reindex(_index)
table1

,Mean \%,Std \%,Skew,Kurt,MD,MD date,\rho,\tau
Future,0.390593,4.631200,-0.505986,4.420370,-26.991962,2020-03-12,1.000000,1.000000
BTC,0.391531,4.402327,-0.585724,4.656525,-25.996530,2020-03-12,0.997513,0.950675
ETH,0.681891,6.010313,-0.255677,5.264618,-32.014398,2020-03-12,0.771182,0.598765
ADA,0.946663,6.699036,0.166123,2.308589,-26.852767,2020-03-12,0.629635,0.482461
LTC,0.322654,6.478058,-0.993548,5.301061,-37.591288,2021-05-19,0.808022,0.611283
XRP,0.298718,7.984279,0.554200,12.488167,-52.765244,2020-12-23,0.451039,0.493882
BITX,0.209250,4.735046,-0.552960,4.038681,-27.021956,2020-03-12,0.976451,0.865932
CRIX,0.220901,4.701095,-0.494949,4.155185,-27.138469,2020-03-12,0.973261,0.849397
BITW100,0.223017,4.764393,-0.629151,4.033059,-27.269408,2020-03-12,0.969868,0.851556
BITW20,0.182461,5.499213,-0.900307,3.718742,-31.009160,2020-03-12,0.784459,0.606355


In [15]:
table2

,ERM,ES1,ES5,VaR1,VaR5,Var
future,0.075553,0.187435,0.123777,0.146793,0.083119,0.002391
bitcoin,0.073294,0.184564,0.120182,0.129051,0.084351,0.002257
eth,0.088572,0.217855,0.143371,0.169576,0.097604,0.003672
ada,0.095791,0.214140,0.152781,0.170823,0.102615,0.004419
ltc,0.096585,0.242580,0.156726,0.186106,0.101043,0.004139
xrp,0.095668,0.259285,0.157976,0.171384,0.104725,0.005142
BITX,0.072262,0.180512,0.120618,0.148662,0.076241,0.002242
CRIX,0.070835,0.181904,0.118451,0.145015,0.074896,0.002210
BITW100,0.073052,0.184992,0.121908,0.147396,0.077242,0.002270
BITW20,0.087091,0.222895,0.140891,0.176662,0.083754,0.003024


In [16]:
np.log(300+4)

5.717027701406222

In [17]:
np.log(300+2)

5.71042701737487

In [18]:
import copulae1 as C
import KDEs as K

In [19]:
kde_rs = K.KDE(rs, 'Gaussian')
kde_rf = K.KDE(rf, 'Gaussian')

C1 = C.Gaussian({'rho':0.9}, kde_rs, kde_rf)
C2 = C.t_Copula({'rho':0.9, 'nu':4}, kde_rs, kde_rf, 2)

NameError: name 'rs' is not defined

In [20]:
u = kde_rs.cdf(rs)
v = kde_rf.cdf(rf)
C1.canonical_calibrate(u,v)
l = np.array([np.log(C1.c(u[i],v[i])) for i in range(len(rs))])
np.sum(l[np.isfinite(l)])

NameError: name 'kde_rs' is not defined

In [21]:
u = kde_rs.cdf(rs)
v = kde_rf.cdf(rf)
C2.canonical_calibrate(u,v)
l = np.array([np.log(C2.c(u[i],v[i])) for i in range(len(rs))])
np.sum(l[np.isfinite(l)])

NameError: name 'kde_rs' is not defined

In [22]:
0.618854*300

185.6562

In [453]:
start

'2019-10-21'

In [454]:
df

,Unnamed: 0,Date,rh,rs,rf,h,copula
0,0,2021-05-27 20:00:00+00:00,-0.020435,-0.012393,0.006583,1.221680,Plackett
1,1,2021-05-26 20:00:00+00:00,0.041045,0.080924,0.032643,1.221680,Plackett
2,2,2021-05-25 20:00:00+00:00,0.057488,-0.018555,-0.062245,1.221680,Plackett
3,3,2021-05-24 20:00:00+00:00,-0.174585,-0.059344,0.094330,1.221680,Plackett
4,4,2021-05-21 20:00:00+00:00,-0.044135,-0.177064,-0.108809,1.221680,Plackett
5,0,2021-05-20 20:00:00+00:00,-0.009896,0.014197,0.021291,1.131641,Plackett
6,1,2021-05-19 20:00:00+00:00,-0.201734,-0.304108,-0.090465,1.131641,Plackett
7,2,2021-05-18 20:00:00+00:00,0.053632,0.031345,-0.019694,1.131641,Plackett
8,3,2021-05-17 20:00:00+00:00,0.251689,0.102715,-0.131645,1.131641,Plackett
9,4,2021-05-14 20:00:00+00:00,0.051885,0.093586,0.036850,1.131641,Plackett
